In [30]:
import tensorflow as tf
from tensorflow import keras

Download model from here: https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/model_2024_hairstyle.keras

In [31]:
model = keras.models.load_model('model_2024_hairstyle.keras')

In [32]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('model_2024_hairstyle.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\E\AppData\Local\Temp\tmpc1i1cx5l\assets


INFO:tensorflow:Assets written to: C:\Users\E\AppData\Local\Temp\tmpc1i1cx5l\assets


Saved artifact at 'C:\Users\E\AppData\Local\Temp\tmpc1i1cx5l'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 200, 200, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  2089318871696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2089318870160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2089318869008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2089318868048: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2089318872272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2089318872464: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [33]:
import tensorflow.lite as tflite

interpreter = tflite.Interpreter(model_path='model_2024_hairstyle.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

In [34]:
input_index, output_index

(0, 13)

In [35]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [36]:
image = download_image('https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg')
image = prepare_image(image, (200, 200))

In [37]:
import numpy as np

def preprocess_input(x):
    x /= 255
    return x

x = np.array(image, dtype='float32')
X = np.array([x])

X = preprocess_input(X)

In [38]:
X.shape

(1, 200, 200, 3)

In [39]:
X[0][0][0]

array([0.23921569, 0.40784314, 0.08627451], dtype=float32)

In [40]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

preds[0].tolist()

[0.8934674859046936]

In [45]:
import requests

url = 'http://localhost:8080/2015-03-31/functions/function/invocations'

data = {'url': 'https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg'}

result = requests.post(url, json=data).json()
print(result)

[0.42985352873802185]
